# Unsupervised Classification of STM

In this notebook we will be applying an unsupervised classification protocol, for the classification of STM images. For the step of feature extraction we will be using both an autoencoder and the VGG-16 pre-trained model. When running this code the user will have to choose one of those to run at a time.

---


In [ ]:
import os
import cv2

import sys
sys.path.append('..')

from pathlib import Path
import pickle

%load_ext autoreload
%autoreload 2
from helper import visualize as vis

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

from keras import models
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Reshape, UpSampling2D, Dropout, Conv2DTranspose, Activation, Concatenate
from keras.models import Model, load_model
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib as mpl

from scipy.stats import mode, stats

import numpy as np
import math
import random

import idx2numpy

from tqdm.auto import tqdm

---

## Step 1: Importing Dataset and Re-shaping images.

---

In [ ]:
### importing STM Images ###

# loading folder
path = "..."
image_folder = [f for f in os.listdir( ... ) if f.endswith((".jpg", ".png", ".jpeg"))]

# ----------- extracting images from folder ----------- #

all_images = []

for i, image_file in enumerate( image_folder ):
    
    image_path = os.path.join( path, image_file )
    img        = cv2.imread( image_path )
    
    all_images.append( img )

all_images = np.array( all_images )

# normalising images
all_images = all_images / np.max(all_images)


In [ ]:
## Reshaping Images ###

# ---------------------------------------------------------- #
def resize_and_convert( images, shape ):
    
    resized_images = tf.image.resize( images , shape)
    
    return resized_images
# ---------------------------------------------------------- #

autoencoder_shape = [64,64]
vgg_shape = [224,224]

# Reshaping images
test_images_auto = resize_and_convert( all_images, autoencoder_shape )
test_images_vgg  = resize_and_convert( all_images, vgg_shape )


---

## Step 2: Loading Model

---

In [ ]:
### ====================== Loading Autoencoder Model ====================== ###

# Uploading Autoencoder model trained for this purpose
model_path = './autoencoder_model_stm'

# loading whole model
entire_model = load_model( model_path )

### ====================== Loading VGG-16 Model =========================== ###

vgg16_path = Path('..','models','VGG16.h5')
if not vgg16_path.is_file():
    vgg16 = keras.applications.VGG16( include_top=True, weights='imagenet' )
    vgg16.save(vgg16_path)
    
else:   
    vgg16 = keras.models.load_model(vgg16_path)


In [ ]:
### =========================== GETTING FEATURE EXTRACTOR =========================== ###

def layer_extractor( layer, model ):
    
    assert layer in [x.name for x in model.layers]  # make sure the layer exists

    new_model = keras.Model(inputs = model.input, outputs=[ model.get_layer( layer ).output ])
    
    return new_model

### ===================== Creating Feature Extractor and feature map ================ ###

# Getting the feature extractor
feature_extractor_auto = layer_extractor('flatten', entire_model)
feature_extractor_vgg  = layer_extractor('fc1',     vgg16 )

# Computing feature map
feature_map_auto = feature_extractor_auto.predict( test_images_auto , verbose=True)
feature_map_vgg = feature_extractor_vgg.predict( test_images_vgg , verbose=True)


---

## Step 3: PCA Components.

- Whitening will be applied.
- $91.89$% variance will be retained.

---

In [ ]:
### Calculating all PCA components

pca_n = PCA(svd_solver='full', whiten=True)

x_pca_ = pca_n.fit_transform( feature_map_vgg )

# Cumulative Variance per component
var_ = pca_n.explained_variance_ratio_.cumsum()

# 92% variance
percentage = 0.9189

components_to_keep = np.where( var_ >= percentage )[0][0] + 1


In [ ]:
### PLOTTING GRAPH: How much variance is kept for a PCA component ###

# Plotting
fig1 = plt.figure(figsize=(15, 5))

# ----------------------------------------------- #
ax1 = fig1.add_subplot(1,2,1)
ax1.plot( range(1,len(var_)+1), var_ , marker='o')

ax1.hlines( y = percentage , xmin=0, xmax = components_to_keep , colors='red', linestyles='dashed', linewidth=2)
ax1.vlines( x = components_to_keep , ymin=0, ymax = percentage , colors='red', linestyles='dashed', linewidth=2)

ax1.set_xscale('log')

ax1.set_xlabel('Number of Components $Log_{10}x$')
ax1.set_ylabel('Cumulative Variance')
ax1.grid(True)

# ----------------------------------------------- #
ax2 = fig1.add_subplot(1,2,2)
ax2.plot( var_ , marker='o')

ax2.hlines( y = percentage , xmin=0, xmax = components_to_keep , colors='red', linestyles='dashed', linewidth=2)
ax2.vlines( x = components_to_keep , ymin=0, ymax = percentage , colors='red', linestyles='dashed', linewidth=2)

ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Variance')
ax2.grid(True)

fig1.suptitle('Cumulative Variance by PCA Components')

print(f'\n{ percentage *100}% variance retained by {components_to_keep} principal components\n')


In [ ]:
### Keeping 50 Components (which retains 92% of variance as discussed) ###

pca_w = PCA( n_components = 50 , svd_solver='full', whiten=True, random_state=123414 )
x_w = pca_w.fit_transform( feature_map_vgg )

pca_nw = PCA( n_components = 50 , svd_solver='full', whiten=False, random_state=123414 )
x_nw = pca_nw.fit_transform( feature_map_vgg )


---

### Visualising Clustering feature map through t-SNE.

---

In [ ]:
### Reducing dimensionality to 2D with tSNE ###

tsne_w  = TSNE( n_components=2, random_state=654753 )
tsne_nw  = TSNE( n_components=2, random_state=654753 )

x_w_tsne  = tsne_w.fit_transform( x_w )
x_nw_tsne  = tsne_w.fit_transform( x_nw )


In [ ]:
### Plotting on a scatter graph ###

fig1 = plt.figure(figsize=(15,5))

# --------------------- Plot 1 --------------------- #
ax1 = fig1.add_subplot(1,2,1)
ax2 = fig1.add_subplot(1,2,2)

ax1.scatter( x_nw_tsne[:, 0], x_nw_tsne[:, 1], marker='o' )
ax1.set_title('Without Whitening')

ax2.scatter( x_w_tsne[:, 0], x_w_tsne[:, 1], marker='o' )
ax2.set_title('With Whitening')

fig1.suptitle('t-SNE Visualization of Image Features');


**Comments**:

- In the cases of both the autoencoder and the vgg-16 model, no whitening provided a clearer feature map via t-SNE for visualisation. Therefore the no whitening feature map will be used for visualisation. However for the actual classification only the whitening is being used.

---

## Step 4: K Means Clustering.

- Value of **k** is set to 4 (based on observations of the data).

---

In [ ]:
### Applying k-means to the Data with whitening applied ###

k = 4

kmeans_w = KMeans( n_clusters = k , init='k-means++', max_iter=500, n_init=500, random_state=213460)
kmeans_w.fit( x_w )

labels_unmatched_w = kmeans_w.labels_ 

print('inertia: {:.2f}'.format(kmeans_w.inertia_))


In [ ]:
### PLOTTING K MEANS CLUSTERING RESULTS ###

colors_array = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

color_labels_w = [colors_array[label] for label in labels_unmatched_w]

# plotting

fig2 = plt.figure(figsize=(15,10))

ax1 = fig2.add_subplot(1,1,1)
scatter1 = ax1.scatter( x_nw_tsne[:, 0], x_nw_tsne[:, 1], c=labels_unmatched_w, cmap='tab10', marker='o' )
ax1.set_title(f'k Means Without Whitening')
cbar1 = plt.colorbar(scatter1, ax=ax1)
cbar1.set_label('Cluster Label')


---

### Further Visualisations

---

In [ ]:

vis.pano_plot(x_nw_tsne[:,0], x_nw_tsne[:,1], all_images, patch_shape=(1, 1))


In [ ]:
### Images in Cluster ###

sides = 6

n_images = int( sides**2 )

cluster_n = 2
positions = np.where( labels_unmatched_w == cluster_n )[0]

# -------------------------------- #

fig1 = plt.figure( figsize=(25,25) )

for i in range( 0, n_images ):

    ax = fig1.add_subplot( sides, sides, i+1 )
    
    ax.imshow( all_images[ positions[i] ] )

plt.show()


---